# Advanced Model with BathNormalization & Adam using keras

The **conv-block-model** looks like: (the shape will be different)
<div align="center">
  <img src="images/model/v2/conv-block.png" height="255" width="1000" /><br>            
</div>

The whole **model** looks like:
<div align="center">
  <img src="images/model/v2/model.png" height="248" width="1000" /><br>            
</div>

The **detail model** :
<div align="center">
  <img src="images/model/v2/model-params.png" height="400" width="800" /><br>            
</div>

In [1]:
import h5py
import numpy as np
from keras import layers
import keras.backend as K
from keras.models import Model
import matplotlib.pyplot as plt
from keras.callbacks import Callback
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D

K.set_image_data_format('channels_last')
%matplotlib tk

Using TensorFlow backend.


## 1. Build the function of defining model

In [2]:
def AdvancedModel(X_shape):
    
    X_input = Input(shape=X_shape, name='X_input')
    
    # 1st CONV-BLOCK
    # Conv2D: strides default (1,1), padding default 'valid'
    # Not use bias but center(offset, or named: 'beta')
    Z1 = Conv2D(filters=8, kernel_size=(3,3), use_bias=False, kernel_initializer='glorot_uniform', name='conv_1')(X_input) 
    Z2 = BatchNormalization(scale=False, moving_variance_initializer='glorot_uniform', name='bn_2')(Z1)
    A = Activation('relu', name='relu_3')(Z2)
    X = AveragePooling2D(pool_size=(3,3), strides=(1,1), name='avg_pool_4')(A)
    
    # 2nd CONV-BLOCK
    Z1 = Conv2D(filters=12, kernel_size=(5,5), use_bias=False, kernel_initializer='glorot_uniform', name='conv_5')(X) 
    Z2 = BatchNormalization(scale=False, moving_variance_initializer='glorot_uniform', name='bn_6')(Z1)
    A = Activation('relu', name='relu_7')(Z2)
    X = AveragePooling2D(pool_size=(5,5), strides=(1,1), name='avg_pool_8')(A)
    
    # 3rd CONV-BLOCK
    Z1 = Conv2D(filters=16, kernel_size=(5,5), use_bias=False, kernel_initializer='glorot_uniform', name='conv_9')(X) 
    Z2 = BatchNormalization(scale=False, moving_variance_initializer='glorot_uniform', name='bn_10')(Z1)
    A = Activation('relu', name='relu_11')(Z2)
    X = AveragePooling2D(pool_size=(5,5), strides=(1,1), name='avg_pool_12')(A)
    
    # 4th CONV-BLOCK
    Z1 = Conv2D(filters=20, kernel_size=(5,5), use_bias=False, kernel_initializer='glorot_uniform', name='conv_13')(X) 
    Z2 = BatchNormalization(scale=False, moving_variance_initializer='glorot_uniform', name='bn_14')(Z1)
    A = Activation('relu', name='relu_15')(Z2)
    X = AveragePooling2D(pool_size=(5,5), strides=(1,1), name='avg_pool_16')(A)
    
    # flatten
    X = Flatten()(X)
    
    # 5th NN-BLOCK
    Z1 = Dense(units=128, use_bias=False, kernel_initializer='glorot_uniform', name='fc_17')(X)
    Z2 = BatchNormalization(scale=False, moving_variance_initializer='glorot_uniform', name='bn_18')(Z1)
    X = Activation('relu', name='relu_19')(Z2)
    
    # 6th NN-BLOCK
    Z1 = Dense(units=64, use_bias=False, kernel_initializer='glorot_uniform', name='fc_20')(X)
    Z2 = BatchNormalization(scale=False, moving_variance_initializer='glorot_uniform', name='bn_21')(Z1)
    X = Activation('relu', name='relu_22')(Z2)
    
    # 7th NN-BLOCK
    Z1 = Dense(units=32, use_bias=False, kernel_initializer='glorot_uniform', name='fc_23')(X)
    Z2 = BatchNormalization(scale=False, moving_variance_initializer='glorot_uniform', name='bn_24')(Z1)
    X = Activation('relu', name='relu_25')(Z2)
    
    # 8th NN-BLOCK
    Y = Dense(units=1, activation='sigmoid', kernel_initializer='glorot_uniform', name='fc_26')(X)
    
    model = Model(inputs = X_input, outputs = Y, name='AdvancedModel')
    
    return model

## 2. Create a callback to record the losses during training.

In [3]:
class LossRecorder(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

## 3. Create a model

In [4]:
model = AdvancedModel(X_shape=(32,32,3))

In [5]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## 4. Load the dataset

In [6]:
with h5py.File('datasets/car-v2.h5', 'r', driver='core') as f:
    DIVIDE = 6400
    X_test, Y_test = f['X'][DIVIDE:], f['Y'][DIVIDE:].reshape(-1, 1)
    X_train, Y_train = f['X'][:DIVIDE], f['Y'][:DIVIDE].reshape(-1, 1)

In [7]:
print('X_shape: ', X_train.shape)
print('Y_shape: ', Y_train.shape)

X_shape:  (6400, 32, 32, 3)
Y_shape:  (6400, 1)


In [8]:
loss_recorder = LossRecorder()

## 5. OK, it's time to train the model!

In [9]:
model.fit(X_train, Y_train, batch_size = 32, epochs = 10, callbacks=[loss_recorder])

Epoch 1/10
6400/6400 [==============================] - 16s - loss: 0.4034 - acc: 0.8342    
Epoch 2/10
6400/6400 [==============================] - 16s - loss: 0.2913 - acc: 0.8842    
Epoch 3/10
6400/6400 [==============================] - 16s - loss: 0.2583 - acc: 0.8958    
Epoch 4/10
6400/6400 [==============================] - 16s - loss: 0.2334 - acc: 0.9077    
Epoch 5/10
6400/6400 [==============================] - 16s - loss: 0.2219 - acc: 0.9133    
Epoch 6/10
6400/6400 [==============================] - 16s - loss: 0.2159 - acc: 0.9163    
Epoch 7/10
6400/6400 [==============================] - 16s - loss: 0.2045 - acc: 0.9186    
Epoch 8/10
6400/6400 [==============================] - 16s - loss: 0.1962 - acc: 0.9209    
Epoch 9/10
6400/6400 [==============================] - 16s - loss: 0.1962 - acc: 0.9216    
Epoch 10/10
6400/6400 [==============================] - 16s - loss: 0.1932 - acc: 0.9234    


## 6. Evaluate the performance of model

In [10]:
preds = model.evaluate(X_test, Y_test)
print ("Final Loss \t = " + str(preds[0]))
print ("Test Accuracy \t = " + str(preds[1]))

800/800 [==============================] - 0s     
Final Loss 	 = 0.235009089112
Test Accuracy 	 = 0.91625


The loss and accuracy are both little worse than training. But it's still ok. 

## 7. Now, let's show the loss.

In [13]:
num = len(loss_recorder.losses)
plt.plot(range(num), loss_recorder.losses)
plt.xlabel('number_of_batch')
plt.ylabel('loss')
plt.show()

<div align="center">
  <img src="images/costs/keras_v2/loss.png" height="500" width="1000" /><br>            
</div>

## 8. Check the model archtecture.

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
X_input (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 30, 30, 8)         216       
_________________________________________________________________
bn_2 (BatchNormalization)    (None, 30, 30, 8)         24        
_________________________________________________________________
relu_3 (Activation)          (None, 30, 30, 8)         0         
_________________________________________________________________
avg_pool_4 (AveragePooling2D (None, 28, 28, 8)         0         
_________________________________________________________________
conv_5 (Conv2D)              (None, 24, 24, 12)        2400      
_________________________________________________________________
bn_6 (BatchNormalization)    (None, 24, 24, 12)        36        
__________

**Also can save the model.**
```
with open('model.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    model.summary(print_fn=lambda x: fh.write(x + '\n'))
```

**save the weights**
```
model.save_weights('trained_params/weights_v2.h5')
```